In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [3]:
train = pd.read_csv("train.csv")
leaderboard = pd.read_csv("leaderboard_dataset.csv")
test = pd.read_csv("test.csv")

In [92]:
y = train["VAR21"]
X = train.drop(["VAR21"], axis=1)
leaderboard
comb = X.append(leaderboard)
comb.iloc[:34000, :]
comb.iloc[34000:, :]
comb['VAR6_1'] = comb['VAR6'].rank()/len(comb['VAR6'])
comb['VAR3_1'] = comb['VAR3'].rank()/len(comb['VAR3'])
comb['VAR12_1'] = comb['VAR12'].rank()/len(comb['VAR12'])
comb['VAR11_1'] = comb['VAR11'].rank()/len(comb['VAR11'])

X = comb.iloc[:34000, :]
leaderboard = comb.iloc[34000:, :]

In [93]:
from sklearn.preprocessing import LabelEncoder

for c in X.columns:
    if X[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(X[c].values) + list(test[c].values) + list(leaderboard[c].values)) 
        X[c] = lbl.transform(list(X[c].values))
        test[c] = lbl.transform(list(test[c].values))
        leaderboard[c] = lbl.transform(list(leaderboard[c].values))

/Users/vrishank/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/vrishank/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [94]:
lbl = LabelEncoder() 
lbl.fit(list(y.values)) 
y = lbl.transform(list(y))

In [95]:
y

array([1, 0, 2, ..., 1, 1, 0])

In [97]:
import xgboost as xgb

# prepare dict of params for xgboost to run with
xgb_params = {
    'n_trees': 700, 
    'eta': 0.01,
    'max_depth': 6,
    'subsample': 0.93,
    'objective': 'multi:softmax',
    'eval_metric': 'mlogloss',
    'silent': 1,
    'num_class' :3
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(X, y)
dtest = xgb.DMatrix(leaderboard)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=1200, # increase to have better results (~700)
                   verbose_eval=50,
                   early_stopping_rounds=50
                  )

num_boost_rounds = len(cv_result)
print('num_boost_rounds=' + str(num_boost_rounds))

# train model
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)


# check f2-score (to get higher score - increase num_boost_round in previous cell)

# make predictions and save results
y_preds = model.predict(dtest)
y_preds

[0]	train-mlogloss:1.09548+4.92702e-05	test-mlogloss:1.09573+1.81904e-05
[50]	train-mlogloss:0.981571+0.00137387	test-mlogloss:0.99371+0.0011203
[100]	train-mlogloss:0.917239+0.0017804	test-mlogloss:0.939877+0.00227647
[150]	train-mlogloss:0.877498+0.0019657	test-mlogloss:0.909705+0.00336758
[200]	train-mlogloss:0.850712+0.00209109	test-mlogloss:0.892069+0.00421924
[250]	train-mlogloss:0.831292+0.00223869	test-mlogloss:0.881333+0.00482168
[300]	train-mlogloss:0.816452+0.00218476	test-mlogloss:0.874592+0.00524562
[350]	train-mlogloss:0.804671+0.00200682	test-mlogloss:0.870406+0.00546259
[400]	train-mlogloss:0.79472+0.00195092	test-mlogloss:0.867683+0.00569533
[450]	train-mlogloss:0.786182+0.00197726	test-mlogloss:0.865938+0.00592527
[500]	train-mlogloss:0.77829+0.00222674	test-mlogloss:0.864774+0.00591357
[550]	train-mlogloss:0.77096+0.00222162	test-mlogloss:0.864018+0.0059661
[600]	train-mlogloss:0.764159+0.00252672	test-mlogloss:0.863466+0.00596393
[650]	train-mlogloss:0.757716+0.0025

array([2., 1., 1., ..., 2., 2., 1.], dtype=float32)

In [98]:
import xgboost as xgb

# prepare dict of params for xgboost to run with
xgb_params = {
    'n_trees': 700, 
    'eta': 0.01,
    'max_depth': 5,
    'subsample': 0.93,
    'objective': 'multi:softmax',
    'eval_metric': 'mlogloss',
    'silent': 1,
    'num_class' :3
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(X, y)
dtest = xgb.DMatrix(leaderboard)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=1200, # increase to have better results (~700)
                   verbose_eval=50,
                   early_stopping_rounds=50
                  )

num_boost_rounds = len(cv_result)
print('num_boost_rounds=' + str(num_boost_rounds))

# train model
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)


# check f2-score (to get higher score - increase num_boost_round in previous cell)

# make predictions and save results
y_preds = model.predict(dtest)
y_preds

[0]	train-mlogloss:1.09567+4.8833e-05	test-mlogloss:1.09581+2.01163e-05


KeyboardInterrupt: 

In [77]:
sub=[]
for i in y_preds:
    if i[0]>i[1] and i[0]>i[2]:
        sub.append("High")
    elif i[1]>i[0] and i[1]>i[2]:
        sub.append("Low")
    else:
        sub.append("Medium")
print(list(y).count(0)/len(y))
print(list(y).count(1)/len(y))
print(list(y).count(2)/len(y))
print(sub.count("High")/len(sub))
print(sub.count("Low")/len(sub))
print(sub.count("Medium")/len(sub))
print(len(sub))

0.11020588235294118
0.3553235294117647
0.5344705882352941
0.0134
0.2208
0.7658
10000


In [78]:
d = {'col1': leaderboard["VAR1"], 'col2': sub}
df = pd.DataFrame(data=d)
import shutil 

filename = "Quant404_IITGuwahati_33"
df.to_csv(filename+'.csv', index=False, header=False)
shutil.copyfile("Quant404_IITGuwahati_22.ipynb", filename+".ipynb")

'Quant404_IITGuwahati_33.ipynb'

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=30000, test_size=4000)


def my_custom_accuracy(y_true, y_pred):
    total=0
    for t,p in zip(y_true, y_pred):
        if(t==p):
            total+=1000
        if(t==0 and p==1):
            total-=100
        if(t==0 and p==2):
            total-=50
        if(t==2 and p==1):
            total-=50
        if(t==2 and p==0):
            total-=100
        if(t==1 and p==2):
            total-=100
        if(t==1 and p==0):
            total-=200
    return total

# Make a custom a scorer from the custom metric function
# Note: greater_is_better=False in make_scorer below would mean that the scoring function should be minimized.
my_custom_scorer = make_scorer(my_custom_accuracy, greater_is_better=True)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=30000, test_size=4000)

#tpot = TPOTClassifier(generations=10, population_size=20, verbosity=3, scoring=my_custom_scorer, cv=3)

In [ ]:
tpot.fit(X_train, y_train)

30 operators have been imported by TPOT.


A Jupyter Widget

Skipped pipeline #14 due to time out. Continuing to the next pipeline.
Skipped pipeline #17 due to time out. Continuing to the next pipeline.
